### When running this notebook via the Galaxy portal
You can access your data via the dataset number. Using a Python kernel, you can access dataset number 42 with ``handle = open(get(42), 'r')``.
To save data, write your data to a file, and then call ``put('filename.txt')``. The dataset will then be available in your galaxy history.
<br><br>Note that if you are putting/getting to/from a different history than your default history, you must also provide the history-id.
<br><br>More information including available galaxy-related environment variables can be found at https://github.com/bgruening/docker-jupyter-notebook. This notebook is running in a docker container based on the Docker Jupyter container described in that link.


In [ ]:
import import_ipynb
import setPath
import ROOT as ROOT
from Scripts.SUSYPheno.bin import bkgjelstenArgReader
import munch as mu
%jsroot off

In [ ]:
from Scripts.SUSYPheno.bin import bkgjelstenArgReader
from Scripts.SUSYPheno.bin.kilelib import AreNumbers, SaveHistory, DictFromTable, DictlistsFromListOfDicts, GetPlainArray, SaveHistory, WriteToFile
from Scripts.SUSYPheno.bin.kilelib_ROOT import NextColourStyle   
from Scripts.SUSYPheno.bin.kilelib_ROOT2 import BLegends
import array

# DESCRIPTION
munch.py (imported above) provides a library which is intended to simplify plotting output from SUSYPheno tool with ROOT.
## Input is either
1. a list of pickle files (each file containing a flat python dict which contains the coordinates and result variables of one point)
2. an ordered table with the header line describing the variable names;
   * each line needs to contain the same number of variables, e.g:
            mass  height  width  varN  var_M 
            2.1   4.2      3      4     4.2 
            3     4.8      2.1    5.3   8   
            8.22  4       34.2    9     9.11
            ... 
    * From this table an x-variable (and y-variable) and a list of result (z) variables are selected.
    * An arbitrary combination of the variables can also be used as variable to the plot, e.g. 
       * var1+var2       
       * var1\*var2
       * 2*var1+var2
       * ...
    
### Examples

```
-coord height             : 1D-plot
-coord height,width       : 2D-plot
-coord height*width,mass  : 2D-plot with height*width as the x-variable
-coord 2.1*height*width,mass  : 2D-plot with 2.1*height*width as the x-variable
-coord height:H,width:W   : 2D-plot with 'height' and 'width' renamed as 'H' and 'W' in the plot
-resvars  var_M           : var_M is the resvar
-resvars  var_M,mass      : var_M and mass are the result variables
```

* For **2D-plots** there will be as many plots as there are result variables.
* For **1D-plots** all the result variables will be plotted in the same canvas.


## OPTIONS :
### Coordinates

```
-coord <varlist>
    -coord mu,M2              : selects these two columns in the input table
    -coord mu:#mu,M2:M_{2}    : renames the axes in the plot
    -coord 10*mu,M2+M1:a_sum  : coordinates can be (complex) combinations of the variables in the table
```

### Variables  

```
-resvars <varlist> : 
    -resvars DM
```

### Plot specifics

```
-title <new canvas title>
-save <filename>     :  name of output file (D='munch.pdf'). If none of the allowed formats (['pdf', 'eps', 'ps', 'jpg', 'gif', 'png']) is explicitly in the name, a '.pdf' is added.
-xrange xmin,xmax    :  uses a different (typically smaller) x-range than what is given in the table.
-yrange ymin,ymax    :  uses a different (typically smaller) y-range than what is given in the table.
-xyrange xymin,xymax : same as above, setting equal range for x and y
-zrange zmin,zmax    : sets min and max for the result variables. Values below/above min/max are then set to min/max
```
All the rangevars above can also be set onesided (min or max) with:

```
-xrangemin / -yrangemin / -xyrangemin / -zrangemin / -xrangemax / -yrangemax / -xyrangemax / -zrangemax
```

### Numbers on plot (2D)
```
--numbers          : adds numbers (not just markers). Format is intuitive.
```
Format of numbers can be changed with:
```
-numbers_angle <angle> : prints numbers at angle <angle> (D=0)
-numbers_size  <size>  : size of numbers (D=0.030)
-numbers_colour <col>  : colour of numbers (D=1=black)
-numbers_asc <angle>,<size>,<col> : specify the three above in one go
-textprecision <0/1/2/3>  : mode for precision in the numbers (D=1)
--numbermarker     : [2D] adds a marker 'o' (instead of number) where the table has a value
-numbermarker <x>  : [2D] adds a marker <x> (instead of number) where the table has a value
```

### Contours (2D)
Can plot contour in any of the variables in the table (on top of the result variable)
```
-cont <var>:<varText>:<value>:<colour>:<style>:<width>
    -cont N1::100            : minimal, only give the var name and the value. Col/style/width default to 1,1,1
    -cont N1:mass_of_N1:100  : replace var name in the contour info put on the plot
    -cont N1:mass_of_N1:100:2:3:4  : also specify colour/style/width of the contour
```
Can have several contours on the same plot (separated by ',,'):
```
-cont <var1><var1Text>:<value>:<colour>:<style>:<width>,,<var2><var2Text>:<value>:<colour>:<style>:<width>
```
### Text
Need to specify <some_text>, typically inside '', then x and y coordinate (in range (0,1))
Optional is to specify size (D=0.05), colour (D=1(=black)) and font
```
-text <some_text>,x,y[,size,col,font]  : Add text to canvas
    -text 'my testplot',0.8,0.2      : minimal
    -text testplot,0.8,0.2,0.3       : size
    -text testplot,0.8,0.2,0.3,2,42  : size,colour,font
    -text testplot,0.8,0.2,0.3,2,42:'some more text',0.3,0.8 : another text, separated by :
```

## MORE OPTIONS:
```
-h                : this help message
--h               : brute info on implemented options
--dict            : dump various default values (which can be changed)
-scale <val>      : scale the result values by <val>
-logx 0/1, --logx : sets log scale on x (D=0)
-logy 0/1, --logy : sets log scale on y (D=0)
-yLogMin <val>    : minimum y axis range (if log)  (D=0.100)
-zLogMin <val>    : minimum z axis range (if log)  (D=0.100)
-rightticky 0/1, --rightticky : get tick marks on right side   (D=1)
-toptickx 0/1, --toptickx     : get tick marks on top   (D=0)
-ticklengthx <val>            : size of ticks along x-axis (D=0.030)
-ticklengthy <val>            : size of ticks along y-axis (D=0.030)
-ticklengthxy <val>           : size of ticks along x- and y-axis (D(x)=0.030, D(y)=0.030)
-gridx <val>, --gridx         : draw grid from x (D=0)
-gridy <val>, --gridy         : draw grid from y (D=0)
-gridxy <val>, --gridxy       : draw grid from x and y (D(x)=0, D(y)=0)
-label_x1 <val>               : size of x-axis labels (D=0.120)
-label_xwid <val>             : (D=0.130) 
-label_y2 <val>               : (D=0.895) 
-label_DY <val>               : (D=0.035)
-label_maxchar <val>          : (D=99) , 
-label_maxcharoverflow <val>  : (D='..')
-topmargin <val>    : change top margin (D=0.10)
-bottommargin <val> : change bottom margin (D=0.10)
-leftmargin <val>   : change left margin (D=0.10)
-rightmargin <val>  : change right margin (D=0.10). Especially it may be relevant to increase the right margin to get the full colour-scale.)
```

## EXAMPLES 2D
1. **Plots mass of N1 in 2D mu-M2 plot using one table**

```
-coord mu,M2  -resvars N1 -fn_table a_table
```
2. **Plots mass of N1 in 2D mu-M2 plot using several tables. Several tables can be combined (if not all the required info is in the same table). Obviously the tables need to be ordered in the same way.**

```
-coord mu,M2  -resvars N2another  -fn_table a_table,another_table  
```

3. **If info is in pickle files**

```
-coord mu,M2  -resvars N2   -ffn_pickle <file_with_list_of_pickles>       
```

4. **Advanced with several contours, titles on axis and using pre-set 2D plotting mode**

```
-coord mu:#mu[GeV],M2:M_{2}[GeV]   -resvars ~N2:N2  --mode2Da  -numbers 0  -vb 1   -cont ~C1::100,200:1:1:4,,~N3::300,400:2:2:2,,~N4::300,400:3:3:3  -fn_table table_elweak.txt
```

## EXAMPLES 1D

1. **Simplest example**

```
-coord mu  -resvars h  -fn_table table1D.txt
```
2. **To specify min/max in plot**

```
-coord mu  -resvars h  -zrange 110,130  -fn_table table1D.txt
```
3. **Redefine text on x-axis**

```
-coord mu:'#mu [GeV]'  -resvars h  -zrange 110,130  -fn_table table1D.txt
```

4. **Set title on z-axis**

```
-coord mu  -resvars h  -zrange 110,130  -ztitle 'm(h) [GeV]'  -fn_table table1D.txt
```

5. **Redefine varname in legend**

```
-coord mu  -resvars h:Higgs  -zrange 110,130   -fn_table table1D.txt
```

6. **Set title on plot**

```
-coord mu  -resvars h  -zrange 110,130  -title 'Higgs mass'  -fn_table table1D.txt
```

7. **Put a marker on datapoints. Try  20-30, 1-7. See [markers.gif](http://root.cern.ch/root/html534/gif/markers.gif)**

```
-coord mu  -resvars h  -zrange 110,130  -msty 20    -fn_table table1D.txt
```

8. **Marker size**

```
-coord mu  -resvars h  -zrange 110,130  -msty 20  -msize 1.8   -fn_table table1D.txt      
```

9. **Marker colour**

```
-coord mu  -resvars h  -zrange 110,130  -msty 20  -msize 1.8  -mcol 3   -fn_table table1D.txt 
```

10. **Marker style,size,colour in one go**

```
-coord mu  -resvars h  -zrange 110,130  -marker_ssc1D 20,1.8,3   -fn_table table1D.txt
```

11. **Specify colour, style and width of line**

```
-coord mu  -resvars h  -zrange 110,130  -lcol 5  -lsty 2  -wid 5   -fn_table table1D.txt 
```

12. **Plot both h and ~N1 in same plot (will automatically get different colours)**

```
-coord mu  -resvars h,~N1  -zrange 60,130   -fn_table table1D.txt
```

13. **Specify colour/style/width of the two graphs**

```
-coord mu  -resvars h,~N1  -zrange 60,130  -lcol 6,8  -lsty 2,9  -lwid 4,2    -fn_table table1D.txt 
```

The following cell defines the plotting. Have a look in the help (above) for more information. The input arguments follows the logic below

In [ ]:
arglist = ['-cont','~C1:DM:125:2:2:2,,h:DM:125:3:2:2',
           '-coord','mu,M2',
           '-resvars', 'h:Higgs mass [GeV]',
           '-fn_table', '/storage/shared/data/fys5555/SUSYPheno/table_elweak_ds.txt',
           '--numbers', '-numbers_size','0.02']
           #'-zrangemin','115',
           #'-zrangemax','130']
try:
    del m
except:
    print("No plot object decleared. Good.")
m = mu.munch(cmd=['ReadArg','PostInit','PrepareXYZ','Main'],optD={'argv':arglist})

## Get the plot ingredients

In [ ]:
# unfortunately the surface/contour plotting of TGRaph2D is not supported in jsroot
%jsroot off

In [ ]:
h = m.getHist()
g = m.get2DCont()
c = m.getCont()
h2 = m.get2DHist()
canv = m.getCanv()

# Work in progress
Unfortunately isn't the _surf_ nor _cont_ supported in jsroot so plots like above can not be made interactively. However, _colz_ option can be used. In all cases zooming doesn't work.

In [ ]:
%jsroot on
can = ROOT.TCanvas()
can.Draw()
#h.Draw("COL Z")
#g[0].GetZaxis().SetRangeUser(-10,120)
print(h2.Integral())
h2.Draw("surf")
g[0].Draw("colz")
#for x in range(1,h2.GetNbinsX()):
#    for y in range(1,h2.GetNbinsY()):
#        print(h2.GetBinContent(x,y))
for cont in c.keys():
    print("contour = %s"%cont)
    #c[cont].Draw("cont4same")